# General instructions

The goal of the project is to materialize a set of **exploratory workloads** over a real-world, large-scale,  open-domain KG: [WikiData](https://www.wikidata.org/wiki/Wikidata:Main_Page)

An exploratory workload  is composed by a set of queries, where each query is related to the information obtained previously.

An exploratory workload starts with a usually vague, open ended question, and does not assume the person issuing the workload has a clear understanding of the data contained in the target database or its structure.

Remeber that:

1. All the queries must run in the python notebook
2. You can use classes and properties only if you find them via a SPARQL query that must be present in the notebook
3. You do not delete useless queries. Keep everything that is synthatically valid 

```
?p <http://schema.org/name> ?name .
```
    
    is the BGP returning a human-readable name of a property or a class in Wikidata.

In [1]:
## SETUP used later

from SPARQLWrapper import SPARQLWrapper, JSON


prefixString = """
##-dbe797b263-##
PREFIX wd: <http://www.wikidata.org/entity/> 
PREFIX wdt: <http://www.wikidata.org/prop/direct/> 
PREFIX sc: <http://schema.org/>
"""

# select and construct queries
def run_query(queryString,verbose = True):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        results = sparql.query()
        json_results = results.convert()
        if len(json_results['results']['bindings'])==0:
            print("Empty")
            return []
        array = []
        for bindings in json_results['results']['bindings']:
            app =  [ (var, value['value'])  for var, value in bindings.items() ] 
            if verbose:
                print( app)
            array.append(app)
        if verbose:
            print(len(array))
        return array

    except Exception as e :
        print("The operation failed", e)
    
# ASk queries
def run_ask_query(queryString):
    to_run = prefixString + "\n" + queryString

    sparql = SPARQLWrapper("http://a256-gc1-03.srv.aau.dk:5820/sparql")
    sparql.setTimeout(300)
    sparql.setReturnFormat(JSON)
    sparql.setQuery(to_run)

    try :
        return sparql.query().convert()

    except Exception as e :
        print("The operation failed", e)

# History Workflow Series ("Cultural Movements explorative search") 

Consider the following exploratory information need:

> investigate the cultural movements, in particular the Renaissance, Baroque and Romanticsm. We'd like to get an overview of the main artists belonging to these movements, musicians, poets, sculptors and painters.

## Useful URIs for the current workflow
The following are given:

| IRI           | Description   | Role      |
| -----------   | -----------   |-----------|
| `wdt:P1647`   | subproperty   | predicate |
| `wdt:P31`     | instance of   | predicate |
| `wdt:P106`    | occupation    | predicate | 
| `wdt:P279`    | subclass      | predicate |
| `wdt:P27`     | nationality   | predicate |
| `wdt:P3342`     | Significant person       | predicate |
| `wd:Q5`| Human       | node |
| `wd:Q37562`| Donatello    | node |
| `wd:Q19675`| Louvre Museum | node |





Also consider

```
wd:Q37562 ?p ?obj .
```

is the BGP to retrieve all **properties of Donatello**

Please consider that when you return a resource, you should return the IRI and the label of the resource. In particular, when the task require you to identify a BGP the result set must always be a list of couples IRI - label.

The workload should:


1. Identify the BGP for Cultural Movement

2. Identify the BGP for Renassaince, Baroque and Romanticism

3. Identify the BGP for composer, painter, sculptor and poet

4. Consider Renaissance, Baroque and Romanticism. Find how many composers, painters, sculptors and poets belong to those cultural movements (the results set must be a list of triples movement, job, #people)

5. Find how many sculptures were made in Italy during Renaissance, Baroque, and Romanticism movements. Please consider all the sculptures and all the sculpture subtypes. (the result set must be a list of triples movement IRI, label and #sculptures).

6. For each European country find how many poets belong to Romanticism (the result set must be a list of triples country IRI, label and #poets).

7. Find the number of sculptures in the Louvre Museum belonging to Renaissance, Baroque, or Romanticism (return a number for each movement) (the result set must be a list of triples movement IRI, label and #sculptures).

## Task 1
Identify the BGP for Cultural Movement

In [2]:
# query example
queryString = """
SELECT DISTINCT ?p ?obj ?pName ?objName
WHERE {
   
   wd:Q37562 ?p ?obj .
   
   ?p sc:name ?pName .
   ?obj sc:name ?objName .
#   FILTER REGEX(?objName, ".*[Cc]utltural.*[Mm]ovement.*") .
}
order by ?objName
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P802'), ('obj', 'http://www.wikidata.org/entity/Q395755'), ('pName', 'student'), ('objName', 'Agostino di Duccio')]
[('p', 'http://www.wikidata.org/prop/direct/P6379'), ('obj', 'http://www.wikidata.org/entity/Q388448'), ('pName', 'has works in the collection'), ('objName', 'Bargello')]
[('p', 'http://www.wikidata.org/prop/direct/P119'), ('obj', 'http://www.wikidata.org/entity/Q573881'), ('pName', 'place of burial'), ('objName', 'Basilica of San Lorenzo')]
[('p', 'http://www.wikidata.org/prop/direct/P6379'), ('obj', 'http://www.wikidata.org/entity/Q573881'), ('pName', 'has works in the collection'), ('objName', 'Basilica of San Lorenzo')]
[('p', 'http://www.wikidata.org/prop/direct/P6379'), ('obj', 'http://www.wikidata.org/entity/Q51177'), ('pName', 'has works in the collection'), ('objName', 'Basilica of Santa Croce')]
[('p', 'http://www.wikidata.org/prop/direct/P9493'), ('obj', 'http://www.wikidata.org/entity/Q59452059'), ('pName', '

In [3]:
# checking if Early Rennosanse has a property that tells us its  a cultural moevement

# Q1472236 = Early Renaissance
queryString = """
SELECT DISTINCT ?p ?obj ?pName ?objName
WHERE {
   
   wd:Q1472236 ?p ?obj .
   
   ?p sc:name ?pName .
   ?obj sc:name ?objName .
#   FILTER REGEX(?objName, ".*[Cc]utltural.*[Mm]ovement.*") .
}
order by ?objName
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P156'), ('obj', 'http://www.wikidata.org/entity/Q1474884'), ('pName', 'followed by'), ('objName', 'High Renaissance')]
[('p', 'http://www.wikidata.org/prop/direct/P155'), ('obj', 'http://www.wikidata.org/entity/Q979160'), ('pName', 'follows'), ('objName', 'Proto-Renaissance')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('obj', 'http://www.wikidata.org/entity/Q4692'), ('pName', 'part of'), ('objName', 'Renaissance')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('obj', 'http://www.wikidata.org/entity/Q968159'), ('pName', 'instance of'), ('objName', 'art movement')]
[('p', 'http://www.wikidata.org/prop/direct/P31'), ('obj', 'http://www.wikidata.org/entity/Q11514315'), ('pName', 'instance of'), ('objName', 'historical period')]
5


In [4]:
# checking if art movement is related to cultural moevement in some way

# Q968159 = art movement

queryString = """
SELECT DISTINCT ?p ?obj ?pName ?objName
WHERE {
   
   wd:Q968159 ?p ?obj .
   
   ?p sc:name ?pName .
   ?obj sc:name ?objName .
#   FILTER REGEX(?objName, ".*[Cc]utltural.*[Mm]ovement.*") .
}
order by ?objName
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('obj', 'http://www.wikidata.org/entity/Q7458526'), ('pName', "topic's main category"), ('objName', 'Category:Art movements')]
[('p', 'http://www.wikidata.org/prop/direct/P1424'), ('obj', 'http://www.wikidata.org/entity/Q14412130'), ('pName', "topic's main template"), ('objName', 'Template:Infobox art movement')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('obj', 'http://www.wikidata.org/entity/Q735'), ('pName', 'subclass of'), ('objName', 'art')]
[('p', 'http://www.wikidata.org/prop/direct/P1889'), ('obj', 'http://www.wikidata.org/entity/Q4502119'), ('pName', 'different from'), ('objName', 'art group')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('obj', 'http://www.wikidata.org/entity/Q2198855'), ('pName', 'subclass of'), ('objName', 'cultural movement')]
[('p', 'http://www.wikidata.org/prop/direct/P2354'), ('obj', 'http://www.wikidata.org/entity/Q1293970'), ('pName', 'has list'), ('objName', 'list of art mo

In [5]:
# art movement is a subclass of cultural movement

# Q2198855 = cultural movement

queryString = """
SELECT DISTINCT ?p ?obj ?pName ?objName
WHERE {
   
   wd:Q2198855 ?p ?obj .
   
   ?p sc:name ?pName .
}
order by ?name
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1687'), ('obj', 'http://www.wikidata.org/entity/P135'), ('pName', 'Wikidata property')]
[('p', 'http://www.wikidata.org/prop/direct/P2581'), ('obj', '00024366n'), ('pName', 'BabelNet ID')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('obj', 'http://www.wikidata.org/entity/Q3533467'), ('pName', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('obj', 'http://www.wikidata.org/entity/Q49773'), ('pName', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('obj', 'Cultural history'), ('pName', 'Commons category')]
[('p', 'http://www.wikidata.org/prop/direct/P5437'), ('obj', '7374'), ('pName', 'EuroVoc ID')]
[('p', 'http://www.wikidata.org/prop/direct/P646'), ('obj', '/m/01s66'), ('pName', 'Freebase ID')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('obj', 'http://www.wikidata.org/entity/Q6638204'), ('pName', "topic's main category")]
8


Final query for this task

In [6]:
# write your final query

In [7]:
# art movement is a subclass of cultural movement

# Q2198855 = cultural movement

queryString = """
SELECT DISTINCT ?p ?obj ?pName ?objName
WHERE {
   
   wd:Q2198855 ?p ?obj .
   
   ?p sc:name ?pName .
}
order by ?name
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1687'), ('obj', 'http://www.wikidata.org/entity/P135'), ('pName', 'Wikidata property')]
[('p', 'http://www.wikidata.org/prop/direct/P2581'), ('obj', '00024366n'), ('pName', 'BabelNet ID')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('obj', 'http://www.wikidata.org/entity/Q3533467'), ('pName', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P279'), ('obj', 'http://www.wikidata.org/entity/Q49773'), ('pName', 'subclass of')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('obj', 'Cultural history'), ('pName', 'Commons category')]
[('p', 'http://www.wikidata.org/prop/direct/P5437'), ('obj', '7374'), ('pName', 'EuroVoc ID')]
[('p', 'http://www.wikidata.org/prop/direct/P646'), ('obj', '/m/01s66'), ('pName', 'Freebase ID')]
[('p', 'http://www.wikidata.org/prop/direct/P910'), ('obj', 'http://www.wikidata.org/entity/Q6638204'), ('pName', "topic's main category")]
8


## Task 2
Identify the BGP for Renassaince, Baroque and Romanticism

In [8]:
# write your queries

In [9]:
# using propert movement to filter for Renassaince, Baroque and Romanticism

# p135 = movement

queryString = """
SELECT distinct ?movement ?name
WHERE {
   
   ?sub wdt:P135 ?movement .
   ?movement sc:name ?name .
   
   {
       Filter Regex(?name, "^[Rr]omanticism.*") .
   }
   UNION
   {
       Filter Regex(?name, "^[Bb]aroque.*") .
   }
   UNION
   {
       Filter Regex(?name, "^[Rr]enaissance.*") .
   }
   
}
order by ?name
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('movement', 'http://www.wikidata.org/entity/Q37853'), ('name', 'Baroque')]
[('movement', 'http://www.wikidata.org/entity/Q911397'), ('name', 'Baroque Revival architecture')]
[('movement', 'http://www.wikidata.org/entity/Q3624106'), ('name', 'Baroque art')]
[('movement', 'http://www.wikidata.org/entity/Q1431376'), ('name', 'Baroque literature')]
[('movement', 'http://www.wikidata.org/entity/Q808561'), ('name', 'Baroque painting')]
[('movement', 'http://www.wikidata.org/entity/Q6076358'), ('name', 'Baroque painting of Spain')]
[('movement', 'http://www.wikidata.org/entity/Q2763408'), ('name', 'Baroque sculpture')]
[('movement', 'http://www.wikidata.org/entity/Q4692'), ('name', 'Renaissance')]
[('movement', 'http://www.wikidata.org/entity/Q502163'), ('name', 'Renaissance Revival architecture')]
[('movement', 'http://www.wikidata.org/entity/Q236122'), ('name', 'Renaissance architecture')]
[('movement', 'http://www.wikidata.org/entity/Q1133779'), ('name', 'Renaissance art')]
[('mo

In [10]:
# we know that:
# Q4692 = Renaissance
# Q37853 = Baroque
# Q37068 = Romanticism


queryString = """
SELECT distinct ?p ?name
WHERE {
   
   {
       wd:Q4692 ?p ?obj .
   }
   UNION
   {
       wd:Q37853 ?p ?obj .
   }
   UNION
   {
       wd:Q37068 ?p ?obj .
   }
   ?p sc:name ?name
   
}
order by ?name
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1014'), ('name', 'Art & Architecture Thesaurus ID')]
[('p', 'http://www.wikidata.org/prop/direct/P7033'), ('name', 'Australian Educational Vocabulary ID')]
[('p', 'http://www.wikidata.org/prop/direct/P508'), ('name', 'BNCF Thesaurus ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2581'), ('name', 'BabelNet ID')]
[('p', 'http://www.wikidata.org/prop/direct/P7832'), ('name', 'Basque Vikidia ID')]
[('p', 'http://www.wikidata.org/prop/direct/P950'), ('name', 'Biblioteca Nacional de España ID')]
[('p', 'http://www.wikidata.org/prop/direct/P268'), ('name', 'Bibliothèque nationale de France ID')]
[('p', 'http://www.wikidata.org/prop/direct/P5019'), ('name', 'Brockhaus Enzyklopädie online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P3241'), ('name', 'Catholic Encyclopedia ID')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('name', 'Commons category')]
[('p', 'http://www.wikidata.org/prop/direct/P935'), ('name', 'Commons galle

Final query for this task

In [11]:
# write your final query

In [12]:
# we know that:
# Q4692 = Renaissance
# Q37853 = Baroque
# Q37068 = Romanticism


queryString = """
SELECT distinct ?p ?name
WHERE {
   
   {
       wd:Q4692 ?p ?obj .
   }
   UNION
   {
       wd:Q37853 ?p ?obj .
   }
   UNION
   {
       wd:Q37068 ?p ?obj .
   }
   ?p sc:name ?name
   
}
order by ?name
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P1014'), ('name', 'Art & Architecture Thesaurus ID')]
[('p', 'http://www.wikidata.org/prop/direct/P7033'), ('name', 'Australian Educational Vocabulary ID')]
[('p', 'http://www.wikidata.org/prop/direct/P508'), ('name', 'BNCF Thesaurus ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2581'), ('name', 'BabelNet ID')]
[('p', 'http://www.wikidata.org/prop/direct/P7832'), ('name', 'Basque Vikidia ID')]
[('p', 'http://www.wikidata.org/prop/direct/P950'), ('name', 'Biblioteca Nacional de España ID')]
[('p', 'http://www.wikidata.org/prop/direct/P268'), ('name', 'Bibliothèque nationale de France ID')]
[('p', 'http://www.wikidata.org/prop/direct/P5019'), ('name', 'Brockhaus Enzyklopädie online ID')]
[('p', 'http://www.wikidata.org/prop/direct/P3241'), ('name', 'Catholic Encyclopedia ID')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('name', 'Commons category')]
[('p', 'http://www.wikidata.org/prop/direct/P935'), ('name', 'Commons galle

## Task 3
Identify the BGP for composer, painter, sculptor and poet

In [13]:
# write your queries

In [14]:
# use property occupation to filter for composer, painter, sculptor and poet

# wdt:P106 = occupation

queryString = """
SELECT distinct ?occupation ?name
WHERE {
   
   ?someperson wdt:P106 ?occupation .
   ?occupation sc:name ?name .
   
   {
       Filter Regex(?name, "^composer") .
   }
   UNION
   {
       Filter Regex(?name, "^painter") .
   }
   UNION
   {
       Filter Regex(?name, "^sculptor") .
   }
   UNION
   {
       Filter Regex(?name, "^poet") .
   }
   
}
#order by ?name
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('occupation', 'http://www.wikidata.org/entity/Q36834'), ('name', 'composer')]
[('occupation', 'http://www.wikidata.org/entity/Q1028181'), ('name', 'painter')]
[('occupation', 'http://www.wikidata.org/entity/Q288728'), ('name', 'painter and varnisher')]
[('occupation', 'http://www.wikidata.org/entity/Q1281618'), ('name', 'sculptor')]
[('occupation', 'http://www.wikidata.org/entity/Q1209498'), ('name', 'poet lawyer')]
[('occupation', 'http://www.wikidata.org/entity/Q482'), ('name', 'poetry')]
[('occupation', 'http://www.wikidata.org/entity/Q49757'), ('name', 'poet')]
[('occupation', 'http://www.wikidata.org/entity/Q1237928'), ('name', 'poet laureate')]
[('occupation', 'http://www.wikidata.org/entity/Q7207537'), ('name', 'poetry reading')]
[('occupation', 'http://www.wikidata.org/entity/Q104824215'), ('name', 'poetry teacher')]
[('occupation', 'http://www.wikidata.org/entity/Q12901590'), ('name', 'poetaster')]
11


In [15]:
#we know that:
# Q36834 = composer
# Q1028181 = painter
# Q1281618 = sculptor
# Q49757 = poet

queryString = """
SELECT distinct ?p ?name
WHERE {
   
   {
       wd:Q36834 ?p ?obj .
   }
   UNION
   {
       wd:Q1028181 ?p ?obj .
   }
   UNION
   {
       wd:Q1281618 ?p ?obj .
   }
   UNION
   {
       wd:Q49757 ?p ?obj .
   }
   
   ?p sc:name ?name
   
}
order by ?name
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P8785'), ('name', 'AniDB tag ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1014'), ('name', 'Art & Architecture Thesaurus ID')]
[('p', 'http://www.wikidata.org/prop/direct/P7033'), ('name', 'Australian Educational Vocabulary ID')]
[('p', 'http://www.wikidata.org/prop/direct/P508'), ('name', 'BNCF Thesaurus ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2581'), ('name', 'BabelNet ID')]
[('p', 'http://www.wikidata.org/prop/direct/P950'), ('name', 'Biblioteca Nacional de España ID')]
[('p', 'http://www.wikidata.org/prop/direct/P268'), ('name', 'Bibliothèque nationale de France ID')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('name', 'Commons category')]
[('p', 'http://www.wikidata.org/prop/direct/P6262'), ('name', 'Fandom article ID')]
[('p', 'http://www.wikidata.org/prop/direct/P646'), ('name', 'Freebase ID')]
[('p', 'http://www.wikidata.org/prop/direct/P7818'), ('name', 'French Vikidia ID')]
[('p', 'http://www.wikida

Final query for this task

In [16]:
# write your final query

In [17]:
#we know that:
# Q36834 = composer
# Q1028181 = painter
# Q1281618 = sculptor
# Q49757 = poet

queryString = """
SELECT distinct ?p ?name
WHERE {
   
   {
       wd:Q36834 ?p ?obj .
   }
   UNION
   {
       wd:Q1028181 ?p ?obj .
   }
   UNION
   {
       wd:Q1281618 ?p ?obj .
   }
   UNION
   {
       wd:Q49757 ?p ?obj .
   }
   
   ?p sc:name ?name
   
}
order by ?name
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P8785'), ('name', 'AniDB tag ID')]
[('p', 'http://www.wikidata.org/prop/direct/P1014'), ('name', 'Art & Architecture Thesaurus ID')]
[('p', 'http://www.wikidata.org/prop/direct/P7033'), ('name', 'Australian Educational Vocabulary ID')]
[('p', 'http://www.wikidata.org/prop/direct/P508'), ('name', 'BNCF Thesaurus ID')]
[('p', 'http://www.wikidata.org/prop/direct/P2581'), ('name', 'BabelNet ID')]
[('p', 'http://www.wikidata.org/prop/direct/P950'), ('name', 'Biblioteca Nacional de España ID')]
[('p', 'http://www.wikidata.org/prop/direct/P268'), ('name', 'Bibliothèque nationale de France ID')]
[('p', 'http://www.wikidata.org/prop/direct/P373'), ('name', 'Commons category')]
[('p', 'http://www.wikidata.org/prop/direct/P6262'), ('name', 'Fandom article ID')]
[('p', 'http://www.wikidata.org/prop/direct/P646'), ('name', 'Freebase ID')]
[('p', 'http://www.wikidata.org/prop/direct/P7818'), ('name', 'French Vikidia ID')]
[('p', 'http://www.wikida

## Task 4
Consider Renaissance, Baroque and Romanticism. Find how many composers, painters, sculptors and poets belong to those cultural movements (the results set must be a list of triples movement, job, #people)

In [18]:
# write your queries

In [19]:
# using propert movemnt to filter and group by on Renaissance, Baroque and Romanticism

# p135 = movement
# Q4692 = Renaissance
# Q37853 = Baroque
# Q37068 = Romanticism

# use property occupation to filter for composer, painter, sculptor and poet

# wdt:P106 = occupation
# Q36834 = composer
# Q1028181 = painter
# Q1281618 = sculptor
# Q49757 = poet

queryString = """
SELECT distinct ?movementName ?occupationName (count(?someperson) as ?numberOfPeople)
WHERE {
   
   ?someperson wdt:P106 ?occupation .
   Filter (?occupation = wd:Q36834 || ?occupation = wd:Q1028181 || ?occupation = wd:Q1281618 || ?occupation = wd:Q49757) .
   ?occupation sc:name ?occupationName .
   
   ?someperson wdt:P135 ?movement .
   Filter (?movement = wd:Q4692 || ?movement = wd:Q37853 || ?movement = wd:Q37068) .
   ?movement sc:name ?movementName .
   
}
group by ?movementName ?occupationName
order by ?movementName 
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('movementName', 'Baroque'), ('occupationName', 'sculptor'), ('numberOfPeople', '328')]
[('movementName', 'Baroque'), ('occupationName', 'poet'), ('numberOfPeople', '39')]
[('movementName', 'Baroque'), ('occupationName', 'painter'), ('numberOfPeople', '378')]
[('movementName', 'Baroque'), ('occupationName', 'composer'), ('numberOfPeople', '6')]
[('movementName', 'Renaissance'), ('occupationName', 'sculptor'), ('numberOfPeople', '18')]
[('movementName', 'Renaissance'), ('occupationName', 'poet'), ('numberOfPeople', '15')]
[('movementName', 'Renaissance'), ('occupationName', 'composer'), ('numberOfPeople', '8')]
[('movementName', 'Renaissance'), ('occupationName', 'painter'), ('numberOfPeople', '56')]
[('movementName', 'Romanticism'), ('occupationName', 'composer'), ('numberOfPeople', '13')]
[('movementName', 'Romanticism'), ('occupationName', 'painter'), ('numberOfPeople', '152')]
[('movementName', 'Romanticism'), ('occupationName', 'sculptor'), ('numberOfPeople', '14')]
[('mov

Final query for this task

In [20]:
# write your final query

In [21]:
# using propert movemnt to filter and group by on Renaissance, Baroque and Romanticism

# p135 = movement
# Q4692 = Renaissance
# Q37853 = Baroque
# Q37068 = Romanticism

# use property occupation to filter for composer, painter, sculptor and poet

# wdt:P106 = occupation
# Q36834 = composer
# Q1028181 = painter
# Q1281618 = sculptor
# Q49757 = poet

queryString = """
SELECT distinct ?movementName ?occupationName (count(?someperson) as ?numberOfPeople)
WHERE {
   
   ?someperson wdt:P106 ?occupation .
   Filter (?occupation = wd:Q36834 || ?occupation = wd:Q1028181 || ?occupation = wd:Q1281618 || ?occupation = wd:Q49757) .
   ?occupation sc:name ?occupationName .
   
   ?someperson wdt:P135 ?movement .
   Filter (?movement = wd:Q4692 || ?movement = wd:Q37853 || ?movement = wd:Q37068) .
   ?movement sc:name ?movementName .
   
}
group by ?movementName ?occupationName
order by ?movementName 
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('movementName', 'Baroque'), ('occupationName', 'sculptor'), ('numberOfPeople', '328')]
[('movementName', 'Baroque'), ('occupationName', 'poet'), ('numberOfPeople', '39')]
[('movementName', 'Baroque'), ('occupationName', 'painter'), ('numberOfPeople', '378')]
[('movementName', 'Baroque'), ('occupationName', 'composer'), ('numberOfPeople', '6')]
[('movementName', 'Renaissance'), ('occupationName', 'sculptor'), ('numberOfPeople', '18')]
[('movementName', 'Renaissance'), ('occupationName', 'poet'), ('numberOfPeople', '15')]
[('movementName', 'Renaissance'), ('occupationName', 'composer'), ('numberOfPeople', '8')]
[('movementName', 'Renaissance'), ('occupationName', 'painter'), ('numberOfPeople', '56')]
[('movementName', 'Romanticism'), ('occupationName', 'composer'), ('numberOfPeople', '13')]
[('movementName', 'Romanticism'), ('occupationName', 'painter'), ('numberOfPeople', '152')]
[('movementName', 'Romanticism'), ('occupationName', 'sculptor'), ('numberOfPeople', '14')]
[('mov

## Task 5
Find how many sculptures were made in Italy during Renaissance, Baroque, and Romanticism movements. Please consider all the sculptures and all the sculpture subtypes. (the result set must be a list of triples movement IRI, label and #sculptures).

In [22]:
# write your queries

In [23]:
# looking for something that is an instance of a sculpture

# wdt:P31 = instance of

queryString = """
SELECT distinct ?sculpture ?name
WHERE {
   
   ?something wdt:P106 ?sculpture .
   ?sculpture sc:name ?name .
   
   {
       Filter Regex(?name, "^sculpture") .
   }
   
}
#order by ?name
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('sculpture', 'http://www.wikidata.org/entity/Q28929702'), ('name', 'sculpture restorer')]
[('sculpture', 'http://www.wikidata.org/entity/Q860861'), ('name', 'sculpture')]
[('sculpture', 'http://www.wikidata.org/entity/Q860828'), ('name', 'sculpture foundry')]
3


In [80]:
# checking if Renaissance is related to italy is some way


# p135 = movement
# Q4692 = Renaissance
# Q37853 = Baroque
# Q37068 = Romanticism

queryString = """
SELECT distinct ?randomSubjectName ?randomPredicate ?p ?obj ?randomPredicateName ?pName ?objName
WHERE {
   
   ?randomSculpture wdt:P31 wd:Q860861 .
   ?randomSculpture ?p ?obj .
   
   ?p sc:name ?pName .
   ?obj sc:name ?objName .
   
   FILTER Regex (?pName, ".*[Cc]ountry.*") .
   FILTER Regex (?objName, ".*[Ii]taly.*") .
   
}
#order by ?objName
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P495'), ('obj', 'http://www.wikidata.org/entity/Q38'), ('pName', 'country of origin'), ('objName', 'Italy')]
[('p', 'http://www.wikidata.org/prop/direct/P17'), ('obj', 'http://www.wikidata.org/entity/Q38'), ('pName', 'country'), ('objName', 'Italy')]
2


In [142]:
# we know that

# Q860861 = sculpture
# wdt:P31 = instance of
# wdt:P279 = subbtype

# P135 = movement
# Q4692 = Renaissance
# Q37853 = Baroque
# Q37068 = Romanticism

# P495 = country of origin
# Q38 = Italy

queryString = """
SELECT distinct ?movement ?movementName (Count(?randomSculpture) as ?numberOfSculptures)
WHERE {

   {
       ?randomSculpture wdt:P31 wd:Q860861 .
   }
   UNION
   {
       ?randomSculpture wdt:P31 ?scultureType .
       ?sculptureType wdt:P279 wd:Q860861
   }
   ?randomSculpture wdt:P495 wd:Q38 .
   
   ?randomSculpture wdt:P135 ?movement .
   {
       ?randomSculpture wdt:P135 ?subMovement .
   }
   UNION
   {
       ?sobMovement wdt:P279 ?movement .
   }
   FILTER (?movement = wd:Q4692 || ?movement = wd:Q37853 || ?movement = wd:Q37068)
   ?movement sc:name ?movementName
   
}
group by ?movement ?movementName
#order by ?objName
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('movement', 'http://www.wikidata.org/entity/Q4692'), ('movementName', 'Renaissance'), ('numberOfSculptures', '3900')]
[('movement', 'http://www.wikidata.org/entity/Q37853'), ('movementName', 'Baroque'), ('numberOfSculptures', '32412')]
[('movement', 'http://www.wikidata.org/entity/Q37068'), ('movementName', 'Romanticism'), ('numberOfSculptures', '60000')]
3


Final query for this task

In [25]:
# write your final query

In [153]:
# we know that

# Q860861 = sculpture
# wdt:P31 = instance of
# wdt:P279 = subbtype

# P135 = movement
# Q4692 = Renaissance
# Q37853 = Baroque
# Q37068 = Romanticism

# P495 = country of origin
# Q38 = Italy

queryString = """
SELECT distinct ?movement ?movementName (Count(?randomSculpture) as ?numberOfSculptures)
WHERE {

   {
       ?randomSculpture wdt:P31 wd:Q860861 .
   }
   UNION
   {
       ?randomSculpture wdt:P31 ?scultureType .
       ?sculptureType wdt:P279 wd:Q860861
   }
   ?randomSculpture wdt:P495 wd:Q38 .
   
   
   {
       ?randomSculpture wdt:P135 ?movement .
   }
   UNION
   {
       ?randomSculpture wdt:P135 ?subMovement .
       ?subMovement wdt:P279 ?movement .
   }
   FILTER (?movement = wd:Q4692 || ?movement = wd:Q37853 || ?movement = wd:Q37068)
   
   ?movement sc:name ?movementName
   
}
group by ?movement ?movementName
#order by ?objName
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('movement', 'http://www.wikidata.org/entity/Q4692'), ('movementName', 'Renaissance'), ('numberOfSculptures', '9150')]
[('movement', 'http://www.wikidata.org/entity/Q37853'), ('movementName', 'Baroque'), ('numberOfSculptures', '2701')]
[('movement', 'http://www.wikidata.org/entity/Q37068'), ('movementName', 'Romanticism'), ('numberOfSculptures', '3000')]
3


## Task 6
For each European country find how many poets belong to Romanticism (the result set must be a list of triples country IRI, label and #poets).

In [26]:
# write your queries

In [91]:
# checking if any propert of italy tells us its in europe

# Q38 = Italy

queryString = """
SELECT DISTINCT ?p ?obj ?pName ?objName
WHERE {
   
   wd:Q38 ?p ?obj .
   
   ?p sc:name ?pName .
   ?obj sc:name ?objName .
   FILTER REGEX(?objName, ".*[Ee]urop.*") .
}
order by ?objName
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P463'), ('obj', 'http://www.wikidata.org/entity/Q8908'), ('pName', 'member of'), ('objName', 'Council of Europe')]
[('p', 'http://www.wikidata.org/prop/direct/P30'), ('obj', 'http://www.wikidata.org/entity/Q46'), ('pName', 'continent'), ('objName', 'Europe')]
[('p', 'http://www.wikidata.org/prop/direct/P361'), ('obj', 'http://www.wikidata.org/entity/Q46'), ('pName', 'part of'), ('objName', 'Europe')]
[('p', 'http://www.wikidata.org/prop/direct/P421'), ('obj', 'http://www.wikidata.org/entity/Q5412122'), ('pName', 'located in time zone'), ('objName', 'Europe/Rome')]
[('p', 'http://www.wikidata.org/prop/direct/P463'), ('obj', 'http://www.wikidata.org/entity/Q141720'), ('pName', 'member of'), ('objName', 'European Air Transport Command')]
[('p', 'http://www.wikidata.org/prop/direct/P463'), ('obj', 'http://www.wikidata.org/entity/Q488981'), ('pName', 'member of'), ('objName', 'European Bank for Reconstruction and Development')]
[('p', 'htt

In [94]:
# does a randomPoet have a propert to show country

# wdt:P106 = occupation
# Q49757 = poet

queryString = """
SELECT DISTINCT ?p ?obj ?pName ?objName
WHERE {

   ?randomPoet wdt:P106 wd:Q49757 .
   ?randomPoet ?p ?obj .
   
   ?p sc:name ?pName .
   ?obj sc:name ?objName .
   FILTER REGEX(?objName, ".*[Ii]taly.*") .
}
order by ?objName
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P19'), ('obj', 'http://www.wikidata.org/entity/Q390241'), ('pName', 'place of birth'), ('objName', 'Bono, Italy')]
[('p', 'http://www.wikidata.org/prop/direct/P166'), ('obj', 'http://www.wikidata.org/entity/Q1497512'), ('pName', 'award received'), ('objName', 'Commemorative Medal of the Unity of Italy')]
[('p', 'http://www.wikidata.org/prop/direct/P102'), ('obj', 'http://www.wikidata.org/entity/Q1280685'), ('pName', 'member of political party'), ('objName', 'Communist Party of Italy')]
[('p', 'http://www.wikidata.org/prop/direct/P551'), ('obj', 'http://www.wikidata.org/entity/Q3155864'), ('pName', 'residence'), ('objName', 'Geographical region of Italy')]
[('p', 'http://www.wikidata.org/prop/direct/P27'), ('obj', 'http://www.wikidata.org/entity/Q38'), ('pName', 'country of citizenship'), ('objName', 'Italy')]
[('p', 'http://www.wikidata.org/prop/direct/P937'), ('obj', 'http://www.wikidata.org/entity/Q38'), ('pName', 'work location'), 

In [112]:
# use property occupation to filter poet
# wdt:P106 = occupation
# Q49757 = poet

# using propert movement to filter  Romanticism
# P135 = movement
# Q37068 = Romanticism

# use property country of origin of poets to group by countries
# P27 = country of citizenship

# use porperty continent of country to filter countries in europe
# P30 = continent
# Q46 = Europe

queryString = """
SELECT distinct ?country ?countryName (count(?somePoet) as ?numberOfPoets)
WHERE {
   
   ?somePoet wdt:P106 wd:Q49757 .
   ?somePoet wdt:P135 wd:Q37068 .
   
   ?somePoet wdt:P27 ?country .
   ?country wdt:P30 wd:Q46 .
   
   ?country sc:name ?countryName .
}
group by ?country ?countryName
order by desc(?numberOfPoets )
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q29'), ('countryName', 'Spain'), ('numberOfPoets', '47')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('countryName', 'France'), ('numberOfPoets', '16')]
[('country', 'http://www.wikidata.org/entity/Q161885'), ('countryName', 'Great Britain'), ('numberOfPoets', '8')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('countryName', 'Germany'), ('numberOfPoets', '7')]
[('country', 'http://www.wikidata.org/entity/Q174193'), ('countryName', 'United Kingdom of Great Britain and Ireland'), ('numberOfPoets', '7')]
[('country', 'http://www.wikidata.org/entity/Q131964'), ('countryName', 'Austrian Empire'), ('numberOfPoets', '6')]
[('country', 'http://www.wikidata.org/entity/Q45'), ('countryName', 'Portugal'), ('numberOfPoets', '4')]
[('country', 'http://www.wikidata.org/entity/Q35'), ('countryName', 'Denmark'), ('numberOfPoets', '4')]
[('country', 'http://www.wikidata.org/entity/Q145'), ('countryName', 'United Kingdom'), ('number

Final query for this task

In [27]:
# write your final query

In [113]:
# use property occupation to filter poet
# wdt:P106 = occupation
# Q49757 = poet

# using propert movement to filter  Romanticism
# P135 = movement
# Q37068 = Romanticism

# use property country of origin of poets to group by countries
# P27 = country of citizenship

# use porperty continent of country to filter countries in europe
# P30 = continent
# Q46 = Europe

queryString = """
SELECT distinct ?country ?countryName (count(?somePoet) as ?numberOfPoets)
WHERE {
   
   ?somePoet wdt:P106 wd:Q49757 .
   ?somePoet wdt:P135 wd:Q37068 .
   
   ?somePoet wdt:P27 ?country .
   ?country wdt:P30 wd:Q46 .
   
   ?country sc:name ?countryName .
}
group by ?country ?countryName
order by desc(?numberOfPoets )
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('country', 'http://www.wikidata.org/entity/Q29'), ('countryName', 'Spain'), ('numberOfPoets', '47')]
[('country', 'http://www.wikidata.org/entity/Q142'), ('countryName', 'France'), ('numberOfPoets', '16')]
[('country', 'http://www.wikidata.org/entity/Q161885'), ('countryName', 'Great Britain'), ('numberOfPoets', '8')]
[('country', 'http://www.wikidata.org/entity/Q183'), ('countryName', 'Germany'), ('numberOfPoets', '7')]
[('country', 'http://www.wikidata.org/entity/Q174193'), ('countryName', 'United Kingdom of Great Britain and Ireland'), ('numberOfPoets', '7')]
[('country', 'http://www.wikidata.org/entity/Q131964'), ('countryName', 'Austrian Empire'), ('numberOfPoets', '6')]
[('country', 'http://www.wikidata.org/entity/Q45'), ('countryName', 'Portugal'), ('numberOfPoets', '4')]
[('country', 'http://www.wikidata.org/entity/Q35'), ('countryName', 'Denmark'), ('numberOfPoets', '4')]
[('country', 'http://www.wikidata.org/entity/Q145'), ('countryName', 'United Kingdom'), ('number

## Task 7
Find the number of sculptures in the Louvre Museum belonging to Renaissance, Baroque, or Romanticism (return a number for each movement) (the result set must be a list of triples movement IRI, label and #sculptures).

In [28]:
# write your queries

In [115]:
# checking how a random sculpture is related to louvre meuseum

# Q860861 = sculpture
# wdt:P31 = instance of
# wd:Q19675 = Louvre Meuseum

queryString = """
SELECT DISTINCT ?p ?obj ?pName ?objName
WHERE {

   ?randomSculpture wdt:P31 wd:Q860861 .
   ?randomSculpture ?p wd:Q19675 .
   
   ?p sc:name ?pName .
   
}
order by ?objName
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('p', 'http://www.wikidata.org/prop/direct/P195'), ('pName', 'collection')]
[('p', 'http://www.wikidata.org/prop/direct/P276'), ('pName', 'location')]
[('p', 'http://www.wikidata.org/prop/direct/P608'), ('pName', 'exhibition history')]
3


In [158]:
# use location propert of sculture to filter louvre meuseum
# use movement propert of sculpture to filter and group by Renaissance, Baroque, or Romanticism

# checking how a random sculpture is related to louvre meuseum

# Q860861 = sculpture
# wdt:P31 = instance of
# wdt:P279 = subclass

# P276 = location
# wd:Q19675 = Louvre Meuseum

# P135 = movement
# Q4692 = Renaissance
# Q37853 = Baroque
# Q37068 = Romanticism


queryString = """
SELECT distinct ?movement ?movementName (Count(?randomSculpture) as ?numberOfSculptures)
WHERE {
   
   {
       ?randomSculpture wdt:P31 wd:Q860861 .
   }
   UNION
   {
       ?randomSculpture wdt:P31 ?scultureType .
       ?sculptureType wdt:P279 wd:Q860861
   }
   
   # sculpture is located in Louvre Meuseum
   ?randomSculpture wdt:P276 wd:Q19675 .
   
   # sculpture belongs to Renaissance, Baroque, or Romanticism or their subclasses
   {
       ?randomSculpture wdt:P135 ?movement .
   }
   UNION
   {
       ?randomSculpture wdt:P135 ?subMovement .
       ?subMovement wdt:P279 ?movement .
   }
   FILTER (?movement = wd:Q4692 || ?movement = wd:Q37853 || ?movement = wd:Q37068).
   
   ?movement sc:name ?movementName .
   
}
group by ?movement ?movementName
#order by ?objName
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('movement', 'http://www.wikidata.org/entity/Q4692'), ('movementName', 'Renaissance'), ('numberOfSculptures', '300')]
1


Final query for this task

In [29]:
# write your final query

In [159]:
# use location propert of sculture to filter louvre meuseum
# use movement propert of sculpture to filter and group by Renaissance, Baroque, or Romanticism

# checking how a random sculpture is related to louvre meuseum

# Q860861 = sculpture
# wdt:P31 = instance of
# wdt:P279 = subclass

# P276 = location
# wd:Q19675 = Louvre Meuseum

# P135 = movement
# Q4692 = Renaissance
# Q37853 = Baroque
# Q37068 = Romanticism


queryString = """
SELECT distinct ?movement ?movementName (Count(?randomSculpture) as ?numberOfSculptures)
WHERE {
   
   {
       ?randomSculpture wdt:P31 wd:Q860861 .
   }
   UNION
   {
       ?randomSculpture wdt:P31 ?scultureType .
       ?sculptureType wdt:P279 wd:Q860861
   }
   
   # sculpture is located in Louvre Meuseum
   ?randomSculpture wdt:P276 wd:Q19675 .
   
   # sculpture belongs to Renaissance, Baroque, or Romanticism or their subclasses
   {
       ?randomSculpture wdt:P135 ?movement .
   }
   UNION
   {
       ?randomSculpture wdt:P135 ?subMovement .
       ?subMovement wdt:P279 ?movement .
   }
   FILTER (?movement = wd:Q4692 || ?movement = wd:Q37853 || ?movement = wd:Q37068).
   
   ?movement sc:name ?movementName .
   
}
group by ?movement ?movementName
#order by ?objName
LIMIT 20
"""

print("Results")
x=run_query(queryString)

Results
[('movement', 'http://www.wikidata.org/entity/Q4692'), ('movementName', 'Renaissance'), ('numberOfSculptures', '300')]
1
